In [ ]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import requests


processor = TrOCRProcessor.from_pretrained('fhswf/TrOCR_Math_handwritten')
model = VisionEncoderDecoderModel.from_pretrained('fhswf/TrOCR_Math_handwritten').to('cuda')



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

In [ ]:
import base64
from google.colab import output
from IPython.display import display, HTML
from PIL import Image
import io
import os

def save_image_data(data_url, filename="mouse_drawing.png"):
  """
  Receives a Base64 encoded data URL and saves it as a PNG image.
  """
  try:
    header, encoded = data_url.split(",", 1)

    decoded_data = base64.b64decode(encoded)

    img = Image.open(io.BytesIO(decoded_data))

    img.save(filename)

    print(f"Successfully saved your drawing as '{filename}'!")
    print("Look for it in the file browser on the left.")

  except Exception as e:
    print(f"Error saving image: {e}")

output.register_callback('notebook.save_image', save_image_data)


html_code = """
<style>
  #canvas-container {
    border: 3px solid #4a4a4a;
    border-radius: 8px;
    padding: 10px;
    display: inline-block;
    background-color: #f0f0f0;
    box-shadow: 2px 2px 10px rgba(0,0,0,0.1);
  }
  canvas {
    border: 1px solid black;
    cursor: crosshair;
    background-color: white;
  }
  .controls {
    margin-top: 10px;
    display: flex;
    align-items: center;
    gap: 15px;
  }
  button {
    padding: 8px 15px;
    border-radius: 5px;
    border: none;
    cursor: pointer;
    font-weight: bold;
  }
  #save-btn { background-color: #4CAF50; color: white; }
  #clear-btn { background-color: #f44336; color: white; }
  label { font-family: sans-serif; font-size: 14px; }
  input[type="color"] {
    width: 40px;
    height: 40px;
    border: none;
    padding: 0;
    cursor: pointer;
  }
</style>

<div id="canvas-container">
  <h3>My Drawing Pad</h3>
  <canvas id="drawing-canvas" width="500" height="400"></canvas>
  <div class="controls">
    <button id="save-btn">Save to Colab</button>
    <button id="clear-btn">Clear Canvas</button>
    <label for="color-picker">Color:</label>
    <input type="color" id="color-picker" value="#000000">
    <label for="brush-size">Size:</label>
    <input type="range" id="brush-size" min="1" max="50" value="5">
  </div>
</div>

<script>
  // Get all our elements from the DOM
  const canvas = document.getElementById('drawing-canvas');
  const ctx = canvas.getContext('2d');
  const saveBtn = document.getElementById('save-btn');
  const clearBtn = document.getElementById('clear-btn');
  const colorPicker = document.getElementById('color-picker');
  const brushSize = document.getElementById('brush-size');
  ctx.fillStyle = 'white';
  ctx.fillRect(0, 0, canvas.width, canvas.height);

  let isDrawing = false;
  let lastX = 0;
  let lastY = 0;

  function draw(e) {
    if (!isDrawing) return; // Stop the function if not drawing

    // Set drawing properties
    ctx.strokeStyle = colorPicker.value;
    ctx.lineWidth = brushSize.value;
    ctx.lineCap = 'round';
    ctx.lineJoin = 'round';

    ctx.beginPath();
    // Start from
    ctx.moveTo(lastX, lastY);
    // Go to
    ctx.lineTo(e.offsetX, e.offsetY);
    ctx.stroke();

    // Update last coordinates
    [lastX, lastY] = [e.offsetX, e.offsetY];
  }

  // Event Listeners
  canvas.addEventListener('mousedown', (e) => {
    isDrawing = true;
    [lastX, lastY] = [e.offsetX, e.offsetY];
  });
  canvas.addEventListener('mousemove', draw);
  canvas.addEventListener('mouseup', () => isDrawing = false);
  canvas.addEventListener('mouseout', () => isDrawing = false); // Stop drawing if mouse leaves canvas

  // Button Listeners
  clearBtn.addEventListener('click', () => {
    ctx.clearRect(0, 0, canvas.width, canvas.height);
  });

  saveBtn.addEventListener('click', () => {
    // Get the canvas content as a data URL (Base64 encoded PNG)
    const dataUrl = canvas.toDataURL('image/png');

    // Call the Python function we registered earlier
    google.colab.kernel.invokeFunction('notebook.save_image', [dataUrl], {});
  });
</script>
"""

display(HTML(html_code))

Successfully saved your drawing as 'mouse_drawing.png'!
Look for it in the file browser on the left.


Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
url = "mouse_drawing.png"
image = Image.open(url).convert("RGB")
pixel_values = processor(images=image, return_tensors="pt").pixel_values.to('cuda')
generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
print(generated_text)

2x+3


In [ ]:
from IPython.display import HTML, display

iframe_code = '<iframe src="https://mathjax.github.io/MathJax-demos-web/input/tex2chtml.html" style="width: 100%; height: 700px; border: 2px solid #007bff; border-radius: 12px; box-shadow: 0 10px 30px rgba(0,0,0,0.15); background-color: white;"></iframe>'

HTML(iframe_code)